In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import boto3
import io
from datetime import date

In [2]:
pd.__version__

'1.3.5'

In [4]:
pd.set_option('display.max_columns', None)

bucketname = 'lems-spot-geodata-dev'
data_folder = 'data/raw'
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucketname)
today = str(date.today())
print(today)
directory_name = today
prefix_objs = bucket.objects.filter(Prefix=f"{data_folder}/{directory_name}/denue")
print(prefix_objs)

prefix_df = []

for obj in prefix_objs:
    key = obj.key
    body = obj.get()['Body'].read()
    print(f'Reading {key}')
    temp = pd.read_csv(io.BytesIO(body), encoding='ISO-8859-1', low_memory=False)
    prefix_df.append(temp)
    
df = pd.concat(prefix_df)
df.info()    


2022-05-24
s3.Bucket.objectsCollection(s3.Bucket(name='lems-spot-geodata-dev'), s3.ObjectSummary)
Reading data/raw/2022-05-24/denue_1.csv
Reading data/raw/2022-05-24/denue_2.csv
Reading data/raw/2022-05-24/denue_3.csv
Reading data/raw/2022-05-24/denue_4.csv
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2236069 entries, 0 to 511658
Data columns (total 42 columns):
 #   Column      Dtype  
---  ------      -----  
 0   id          int64  
 1   clee        object 
 2   nom_estab   object 
 3   raz_social  object 
 4   codigo_act  int64  
 5   nombre_act  object 
 6   per_ocu     object 
 7   tipo_vial   object 
 8   nom_vial    object 
 9   tipo_v_e_1  object 
 10  nom_v_e_1   object 
 11  tipo_v_e_2  object 
 12  nom_v_e_2   object 
 13  tipo_v_e_3  object 
 14  nom_v_e_3   object 
 15  numero_ext  float64
 16  letra_ext   object 
 17  edificio    object 
 18  edificio_e  object 
 19  numero_int  float64
 20  letra_int   object 
 21  tipo_asent  object 
 22  nomb_asent  object 
 23  

In [8]:
df = df.drop(columns=['clee', 'raz_social', 'codigo_act', 'nombre_act', 'tipo_v_e_1', 'nom_v_e_1', 'tipo_v_e_2', 'nom_v_e_2', 'tipo_v_e_3', 'nom_v_e_3', 'edificio', 'edificio_e', 'ageb', 'manzana', 'tipoUniEco'])
df.head()

,id,nom_estab,per_ocu,tipo_vial,nom_vial,numero_ext,letra_ext,numero_int,letra_int,tipo_asent,nomb_asent,tipoCenCom,nom_CenCom,num_local,cod_postal,cve_ent,entidad,cve_mun,municipio,cve_loc,localidad,telefono,correoelec,www,latitud,longitud,fecha_alta
0,45602,AABARROTES LOS PINOS,0 a 5 personas,CALLE,SIERRA DE LA GAVIA,221.0,NaN,NaN,NaN,FRACCIONAMIENTO,LAS CUMBRES,NaN,NaN,NaN,20175.0,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes ...,NaN,NaN,NaN,21.911178,-102.256462,2014-12
1,9340534,ABAROTES ESCOBEDO,0 a 5 personas,CALLE,ING. MIGUEL ANGEL BARBERENA VEGA,518.0,NaN,NaN,NaN,FRACCIONAMIENTO,MUNICIPIO LIBRE,NaN,NaN,NaN,NaN,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes ...,NaN,NaN,NaN,21.895504,-102.266840,2021-05
2,35543,ABAROTES MARCOS,0 a 5 personas,CALLE,SARGENTO LIBERATO SANTA CRUZ,604.0,NaN,NaN,NaN,COLONIA,GREMIAL,NaN,NaN,NaN,20030.0,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes ...,NaN,NaN,NaN,21.893444,-102.291618,2014-12
3,20536,ABAROTES MORALES,0 a 5 personas,CALLE,COSIO SUR,117.0,NaN,0.0,NaN,COLONIA,ZONA CENTRO,NaN,NaN,NaN,20000.0,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes ...,4499154508,NaN,NaN,21.881855,-102.286655,2010-07
4,8797198,ABAROTES ROQUE CORDOVA,0 a 5 personas,CALLE,PIÑA,106.0,1,NaN,NaN,LOCALIDAD,CANADA GRANDE DE COTORINA,NaN,NaN,NaN,20394.0,1,Aguascalientes,1,Aguascalientes,125,Cañada Grande de Cotorina ...,NaN,NaN,NaN,21.782423,-102.237041,2019-11


In [9]:
df['fecha_alta'].value_counts()

2010-07    847516
2019-11    844698
2014-12    431107
2020-11     23223
2021-05     14923
2020-04     11469
2016-01     10717
2019-04      9853
2021-11      7012
2013-07      6109
2011-03      5330
2017-03      4923
2016-10      4640
2012-06      4274
2018-11      4049
2017-11      3710
2018-03      2282
2017 03       124
2013 07       104
2012-07         5
2013-08         1
Name: fecha_alta, dtype: int64

In [21]:
df1 = df[df['nom_estab'].str.contains("PIZZERIA")]
df2 = df[df['nom_estab'].str.contains("PIZZA")]
frames = [df1, df2]
df3 = pd.concat(frames)

In [22]:
df3['nom_estab'].unique()

array(['PIZZERIA PEPE S', 'ABARROTES  PIZZERIA Y PANADERIA',
       'SUPER EXPRES Y PIZZERIA', 'TIENDA DE ABARROTES SAY Y PIZZERIA',
       'TIENDA Y PIZZERIA GEMA', 'PANADERIA Y PIZZERIA BETOS',
       'PASTELERIA Y PIZZERIA BUENAVISTA', 'PIZZERIA SIN NOMBRE',
       'PIZZERIA Y PANADERIA LA DELICIA', 'PANDERIA Y PIZZERIA',
       'PIZZERIA TAKE', 'PLANET PIZZA', 'PIZZAS PAPALUAPAN',
       'ABARROTES Y PIZZAS HERMANOS MORALES HERNÁNDEZ',
       'BODEGA DE CAJAS PIZZAS GUEROS', 'QUICKTIMEPIZZA',
       'TIENDA PIZZARRO', 'MISCELANEA LA SABROZAS PIZZAS',
       'LITTLE CAESARS PIZZA', 'QUESSI PIZZAS', 'TOTOÑON POLLO Y PIZZA',
       'OAN Y PIZZA', 'GUEROS PIZZA', 'PASTEL Y PIZZA',
       'PRODUCTOS PARA PIZZAS SA DE CV', 'FITURAS Y PIZZAS SIN NOMBRE',
       'PIZZAS', 'DULCES Y VENTA DE PIZZA SIN NOMBRE',
       'ELABORACIÓN DE PIZZAS Y FRITURAS SIN NOMBRE', 'PIZZAS ANGELO',
       'PIZZAS LA HAWAIANA', 'PIZZAS MI QUERIDO TEQUIX',
       'PIZZAS Y ALGO MAS', 'CARNICERIA PIZZAS',
      

In [23]:
df3['nom_estab'].value_counts()

LITTLE CAESARS PIZZA                             4
PIZZERIA PEPE S                                  1
DULCES Y VENTA DE PIZZA SIN NOMBRE               1
PIZZAS ANGELO                                    1
PIZZAS LA HAWAIANA                               1
PIZZAS MI QUERIDO TEQUIX                         1
PIZZAS Y ALGO MAS                                1
CARNICERIA PIZZAS                                1
VENTA DE DULCES REFRESCOS Y PIZZA.               1
VENTA DE PAN VERDURA Y ELABORACION DE PIZZAS     1
CONELI PIZZA Y PASTA                             1
DEPOSITO LA PIZZA                                1
VENTA DE PIZZA BAGET CUERNOS                     1
PEDROS PIZZAS                                    1
PIZZAS ANGELLYS                                  1
DILCERIA Y PIZZA UNION                           1
PASTELERIA Y VENTA DE PIZZA DAUZON               1
PIZZAS Y POSTRES TORRES                          1
QUESOS Y PIZZAS LA PAZ                           1
PIPERPAIPER PIZZA REPARACION  D

In [24]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55 entries, 4122 to 455926
Data columns (total 27 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          55 non-null     int64  
 1   nom_estab   55 non-null     object 
 2   per_ocu     55 non-null     object 
 3   tipo_vial   55 non-null     object 
 4   nom_vial    55 non-null     object 
 5   numero_ext  38 non-null     float64
 6   letra_ext   34 non-null     object 
 7   numero_int  19 non-null     float64
 8   letra_int   3 non-null      object 
 9   tipo_asent  55 non-null     object 
 10  nomb_asent  55 non-null     object 
 11  tipoCenCom  1 non-null      object 
 12  nom_CenCom  1 non-null      object 
 13  num_local   1 non-null      object 
 14  cod_postal  55 non-null     float64
 15  cve_ent     55 non-null     int64  
 16  entidad     55 non-null     object 
 17  cve_mun     55 non-null     int64  
 18  municipio   55 non-null     object 
 19  cve_loc     55 non-null 

In [ ]:
sns.heatmap(df.isnull(), yticklabels = False, cbar = False, cmap = "Blues")